In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [2]:
df_term=pd.read_csv('../Data/hiring_data.csv')
df_term.head()

,Last Name,First Name,Hire Date,Termination Date,Years of Current Service,Tenure Category,Primary Termination Category,Primary Termination Reason,Secondary Termination Reasons,Involuntary Termination,...,Location,Company,Organization,Manager,HR Business Partner,Cost Center,Job Family Base,Job Family,Management Level,Job Classification
0,Tavelli,Matthew,1/18/2010,4/6/2018,8.22,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Contract Loss,NaN,NaN,...,"USA, MO, Kansas City (2300 Main St)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Darin Morris (528274),NaN,18 ARTS - Non Client Site,Technology,SD Project/Program Manager,Associate,2 - Professionals (EEO-1 Job Categories-United...
1,Hartwell,Andrew,7/24/2017,4/6/2018,0.70,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,"USA, MD, Rockville (One Preserve Pkwy)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Saifur Rahman (537235),NaN,14 Risk Assessment - Non Client Site,Technology,Software Engineering,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...
2,Vadicherla,Sudhakar,5/30/2017,4/6/2018,0.85,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Family Reasons,Terminate Employee > Voluntary > Benefits,NaN,...,"USA, VA, Herndon (575 Herndon Pkwy)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Christopher Cullerot (574970),NaN,14 Risk Assessment - Non Client Site,Technology,SD Data Engineer,Associate,2 - Professionals (EEO-1 Job Categories-United...
3,Barke,Jakob,5/31/2016,4/13/2018,1.87,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,"USA, MD, Lanham (5000 Ellin Rd)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,George Filippakis | Nick Filippakis (581389),NaN,23 Change Technology - Client Site,Technology,SD Project/Program Manager,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...
4,Zewdie,Kaleb,8/29/2016,4/13/2018,1.62,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Relocation,NaN,NaN,...,"USA, MD, Bethesda (9000 Rockville Pike)",631 Booz Allen Hamilton_United States,Booz Allen Hamilton Inc. (Horacio Rozanski (20...,Scot McGinn (520950),NaN,26 Op & Support Analysis - Client Site,Technology,Software Engineering,Associate,2 - Professionals (EEO-1 Job Categories-United...


In [3]:
df_term['Hire Date'] = pd.to_datetime(df_term['Hire Date'])
df_term['Termination Date'] = pd.to_datetime(df_term['Termination Date'])

df_term['Hire Date Month'] = pd.to_datetime(df_term['Hire Date']).dt.to_period('M')
df_term['Termination Date Month'] = pd.to_datetime(df_term['Termination Date']).dt.to_period('M')

df_term['Hire Date Year'] = pd.to_datetime(df_term['Hire Date']).dt.to_period('Y')
df_term['Termination Date Year'] = pd.to_datetime(df_term['Termination Date']).dt.to_period('Y')

In [4]:
df_term.head()

,Last Name,First Name,Hire Date,Termination Date,Years of Current Service,Tenure Category,Primary Termination Category,Primary Termination Reason,Secondary Termination Reasons,Involuntary Termination,...,HR Business Partner,Cost Center,Job Family Base,Job Family,Management Level,Job Classification,Hire Date Month,Termination Date Month,Hire Date Year,Termination Date Year
0,Tavelli,Matthew,2010-01-18,2018-04-06,8.22,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Contract Loss,NaN,NaN,...,NaN,18 ARTS - Non Client Site,Technology,SD Project/Program Manager,Associate,2 - Professionals (EEO-1 Job Categories-United...,2010-01,2018-04,2010,2018
1,Hartwell,Andrew,2017-07-24,2018-04-06,0.70,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,NaN,14 Risk Assessment - Non Client Site,Technology,Software Engineering,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...,2017-07,2018-04,2017,2018
2,Vadicherla,Sudhakar,2017-05-30,2018-04-06,0.85,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Family Reasons,Terminate Employee > Voluntary > Benefits,NaN,...,NaN,14 Risk Assessment - Non Client Site,Technology,SD Data Engineer,Associate,2 - Professionals (EEO-1 Job Categories-United...,2017-05,2018-04,2017,2018
3,Barke,Jakob,2016-05-31,2018-04-13,1.87,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Interests Did...,NaN,NaN,...,NaN,23 Change Technology - Client Site,Technology,SD Project/Program Manager,Senior Consultant,2 - Professionals (EEO-1 Job Categories-United...,2016-05,2018-04,2016,2018
4,Zewdie,Kaleb,2016-08-29,2018-04-13,1.62,NaN,Terminate Employee > Voluntary,Terminate Employee > Voluntary > Relocation,NaN,NaN,...,NaN,26 Op & Support Analysis - Client Site,Technology,Software Engineering,Associate,2 - Professionals (EEO-1 Job Categories-United...,2016-08,2018-04,2016,2018


In [5]:
df_term['Full Name']=df_term['First Name'] + '' + df_term['Last Name']

In [6]:
df_hire=pd.read_csv('../Data/turnover_data.csv')
df_hire.head()

,Hire Date,Legal Name,Job Requisition,Create Job Requisition Reason,Hiring Manager,Recruiter,Principal,Job Profile,Management Level,Location,Job Requisition Eligibility Agency,Eligibility,Job Application Source,Source Category,Requisition Type
0,5/7/2018,"Bruce, Alexander H","R0025328 Digital Strategist, Mid (Filled)",Recruiting > New Requisition,Nishana Kuruppu (553726),NaN,Nancy Rhodes (541081),Digital IT Strategist Mid (inactive),Senior Consultant,"USA, DC, Washington (901 15th St NW)",None,None,Other,Booz Allen Hamilton websites,Sold and Funded
1,5/7/2018,"Raheem, Lynette J",R0022859 Privacy Engineer (Filled),Recruiting > New Requisition,Paul Simonoff (575378),NaN,NaN,Cyber Sec Pol & Comp Analyst Sr (inactive),Associate,"USA, DC, Washington (901 15th St NW)",DOD,None,Other,Job Board,Sold and Funded
2,5/21/2018,"Reed, Erin",R0027023 Energy Communications Specialist (Fil...,Recruiting > New Requisition,Lisa Brundage (580564),Shelby Lancaster (603308),Scott Thigpen (528431),Communications Spec Mid (inactive),Senior Consultant,"USA, VA, Arlington (1550 Crystal Dr Suite 300)...",DOD,Secret,Other,Job Board,Sold and Funded
3,5/29/2018,"Feather, Shellie",R0027563 Energy Performance Contracting Projec...,Recruiting > New Requisition,Christopher Watroba (544608),Shelby Lancaster (603308),NaN,Project Manager Sr (inactive),Associate,"USA, WA, Seattle (Home)",DOD,None,Other,Booz Allen Hamilton websites,Sold and Funded
4,6/4/2018,"Nayudu, Anupama",R0022502 Oracle Database Administrator (Filled),Recruiting > New Requisition,Oanh Dang (525109),Leroy Gifford | Roy Gifford (524804),James Hurst (534911),Database Admin Sr (inactive),Associate,"USA, VA, Herndon (575 Herndon Pkwy)",DOD,None,Referral,Referral,Sold and Funded


In [7]:
df_hire['Hire Date'] = pd.to_datetime(df_hire['Hire Date'])
df_hire['Hire Date Month'] = pd.to_datetime(df_hire['Hire Date']).dt.to_period('M')

In [8]:
time_series_term=df_term.groupby('Termination Date Month')['Full Name'].count()
time_series_term

Termination Date Month
2018-04    16
2018-05    31
2018-06    21
2018-07    26
2018-08    25
2018-09    14
2018-10    22
2018-11    13
2018-12     5
2019-01    19
2019-02    19
2019-03    16
2019-04    15
2019-05    22
2019-06    20
2019-07    29
2019-08    28
2019-09    17
2019-10    13
2019-11    22
2019-12    15
2020-01    26
2020-02    21
2020-03    17
2020-04    14
Freq: M, Name: Full Name, dtype: int64

In [10]:
df_unemployment=pd.read_csv('../Data/umemployment.csv')
df_unemployment.head
df_unemployment.columns  


Index(['DATE', 'UNRATE'], dtype='object')

In [11]:
df_unemployment['DATE'] = pd.to_datetime(df_unemployment['DATE'])
df_unemployment['DATE Month'] = pd.to_datetime(df_unemployment['DATE']).dt.to_period('M')
df_unemployment.set_index('DATE Month',inplace=True)
df_unemployment

,DATE,UNRATE
DATE Month,,
2018-04,2018-04-01,4.0
2018-05,2018-05-01,3.8
2018-06,2018-06-01,4.0
2018-07,2018-07-01,3.8
2018-08,2018-08-01,3.8
2018-09,2018-09-01,3.7
2018-10,2018-10-01,3.8
2018-11,2018-11-01,3.7
2018-12,2018-12-01,3.9


In [15]:
df_claims=pd.read_csv('../Data/Number_of_claims.csv')
df_claims.columns

Index(['DATE', 'CCSA'], dtype='object')

In [16]:
df_claims['DATE Month'] = pd.to_datetime(df_claims['DATE']).dt.to_period('M')
df_claims.set_index('DATE Month',inplace=True)
df_claims

,DATE,CCSA
DATE Month,,
2018-04,4/7/2018,1864000
2018-04,4/14/2018,1824000
2018-04,4/21/2018,1778000
2018-04,4/28/2018,1787000
2018-05,5/5/2018,1743000
2018-05,5/12/2018,1747000
2018-05,5/19/2018,1747000
2018-05,5/26/2018,1750000
2018-06,6/2/2018,1720000


In [18]:
time_series_hires=df_term.groupby('Hire Date Month')['Full Name'].count()


In [19]:
time_series_claims=df_claims.groupby('DATE Month')['CCSA'].count()

In [20]:
time_series_hires=df_hire.groupby('Hire Date Month')['Legal Name'].count()
time_series_hires

Hire Date Month
2018-05     4
2018-06     8
2018-07     7
2018-08     8
2018-09    11
2018-10    16
2018-11     9
2018-12     5
2019-01     5
2019-02    24
2019-03    22
2019-04    27
2019-05     9
2019-06    16
2019-07    22
2019-08    13
2019-09    13
2019-10     6
2019-11    11
2019-12    14
2020-01     7
2020-02     4
2020-03     3
2020-04     3
Freq: M, Name: Legal Name, dtype: int64

In [21]:
full_time_series_df_non_un=pd.merge(time_series_term,time_series_hires,how='outer',left_index=True, right_index=True)

In [22]:
full_time_series_df_non_un_1=pd.merge(full_time_series_df_non_un,df_unemployment,how='outer',left_index=True, right_index=True)

In [28]:
full_time_series_df=pd.merge(full_time_series_df_non_un_1,time_series_claims,how='outer',left_index=True, right_index=True)

In [29]:
full_time_series_df.fillna(0,inplace=True)

In [30]:
full_time_series_df

,Full Name,Legal Name,DATE,UNRATE,CCSA
Termination Date Month,,,,,
2018-04,16,0.0,2018-04-01,4.0,4
2018-05,31,4.0,2018-05-01,3.8,4
2018-06,21,8.0,2018-06-01,4.0,5
2018-07,26,7.0,2018-07-01,3.8,4
2018-08,25,8.0,2018-08-01,3.8,4
2018-09,14,11.0,2018-09-01,3.7,5
2018-10,22,16.0,2018-10-01,3.8,4
2018-11,13,9.0,2018-11-01,3.7,4
2018-12,5,5.0,2018-12-01,3.9,5


In [31]:
full_time_series_df.columns=['termination','hires','date_unemployment','unemployment','total_claims']

In [32]:
full_time_series_df.head()

,termination,hires,date_unemployment,unemployment,total_claims
Termination Date Month,,,,,
2018-04,16,0.0,2018-04-01,4.0,4
2018-05,31,4.0,2018-05-01,3.8,4
2018-06,21,8.0,2018-06-01,4.0,5
2018-07,26,7.0,2018-07-01,3.8,4
2018-08,25,8.0,2018-08-01,3.8,4


In [33]:
full_time_series_df.to_csv('time_series_forecast_day.csv')

In [35]:
full_time_series_df.drop('date_unemployment',axis=1,inplace=True)
full_time_series_df = full_time_series_df.iloc[1:]

In [36]:
full_time_series_df

,termination,hires,unemployment,total_claims
Termination Date Month,,,,
2018-05,31,4.0,3.8,4
2018-06,21,8.0,4.0,5
2018-07,26,7.0,3.8,4
2018-08,25,8.0,3.8,4
2018-09,14,11.0,3.7,5
2018-10,22,16.0,3.8,4
2018-11,13,9.0,3.7,4
2018-12,5,5.0,3.9,5
2019-01,19,5.0,4.0,4


In [37]:
full_time_series_df.to_csv('time_series_forecast.csv')

In [173]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=7
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """Check Granger Causality of all possible combinations of the Time series.
    The rows are the response variable, columns are predictors. The values in the table 
    are the P-Values. P-Values lesser than the significance level (0.05), implies 
    the Null Hypothesis that the coefficients of the corresponding past values is 
    zero, that is, the X does not cause Y can be rejected.

    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(full_time_series_df, variables = full_time_series_df.columns)    

,termination_x,hires_x,unemployment_x
termination_y,1.0,0.0,0.0
hires_y,0.0,1.0,0.0
unemployment_y,0.0,0.0,1.0


In [175]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(full_time_series_df)

LinAlgError: Matrix is not positive definite

In [176]:
import plotly.express as px

fig = px.line(full_time_series_df, y="termination")
fig.show()

In [177]:
fig = px.line(full_time_series_df, y="hires")
fig.show()

In [178]:
nobs = 4
df_train, df_test = full_time_series_df[0:-nobs], full_time_series_df[-nobs:]

# Check size
print(df_train.shape)  # (119, 8)
print(df_test.shape)  # (4, 8)

(20, 3)
(4, 3)


In [191]:
df_test

,termination,hires,unemployment
Termination Date Month,,,
2020-01,26,7.0,3.6
2020-02,21,4.0,3.5
2020-03,17,3.0,4.4
2020-04,14,3.0,14.7


In [179]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

In [180]:
# ADF Test on each column
for name, column in df_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "termination" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.6227
 No. Lags Chosen       = 7
 Critical value 1%     = -4.138
 Critical value 5%     = -3.155
 Critical value 10%    = -2.714
 => P-Value = 0.0884. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "hires" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.5515
 No. Lags Chosen       = 7
 Critical value 1%     = -4.138
 Critical value 5%     = -3.155
 Critical value 10%    = -2.714
 => P-Value = 0.5079. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "unemployment" 
    -----------------------------------------------
 Null Hypothesis: 

In [181]:
model = VAR(df_train)
for i in [1,2,3,4,5]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  3.6966349752749728
BIC :  4.293122751590619
FPE :  41.09182205825668
HQIC:  3.7975844062502344 

Lag Order = 2
AIC :  4.050943440145344
BIC :  5.089710491024203
FPE :  65.40192962658229
HQIC:  4.19417540968259 

Lag Order = 3
AIC :  4.053916467326576
BIC :  5.524292956837546
FPE :  96.95867467704542
HQIC:  4.2000747900610955 

Lag Order = 4
AIC :  0.7880080064225785
BIC :  2.671193016882045
FPE :  15.166038674849647
HQIC:  0.8844425290464315 

Lag Order = 5
AIC :  -175.28344095434804
BIC :  -173.01768031082096
FPE :  -3.7150889957153974e-75
HQIC:  -175.30757603945912 



In [182]:
x = model.select_order(maxlags=5)
x.summary()

,AIC,BIC,FPE,HQIC
0,3.762,3.904,43.06,3.760
1,3.695,4.262,41.88,3.689
2,4.231,5.222,86.96,4.220
3,4.387,5.803,184.0,4.371
4,-30.94,-29.10,5.511e-13,-30.96
5,-175.3*,-173.0*,-3.715e-75*,-175.3*


In [183]:
model_fitted = model.fit(2)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Mon, 11, May, 2020
Time:                     12:43:52
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                    5.08971
Nobs:                     18.0000    HQIC:                   4.19418
Log likelihood:          -92.0812    FPE:                    65.4019
AIC:                      4.05094    Det(Omega_mle):         24.4111
--------------------------------------------------------------------
Results for equation termination
                     coefficient       std. error           t-stat            prob
----------------------------------------------------------------------------------
const                  26.990477        57.221974            0.472           0.637
L1.termination          0.247464         0.286192            0.865           0.387
L1.hires                0.235512         0.284

In [184]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_train.values[-lag_order:]
forecast_input

2


array([[22. , 11. ,  3.5],
       [15. , 14. ,  3.5]])

In [185]:
# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df_train.index[-nobs:], columns=df_train.columns + '_2d')
df_forecast

,termination_2d,hires_2d,unemployment_2d
Termination Date Month,,,
2019-09,19.655129,10.054448,3.567870
2019-10,20.165957,12.756465,3.562407
2019-11,20.375529,11.946757,3.574576
2019-12,20.085790,11.695120,3.576365


In [186]:
def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df_train[col].iloc[-1]-df_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
        # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df_train[col].iloc[-1] + df_fc[str(col)+'_1d'].cumsum()
    return df_fc

In [187]:
df_results = invert_transformation(df_train, df_forecast, second_diff=True)        
df_results.loc[:, ['termination_2d', 'hires_2d','unemployment_2d']]

,termination_2d,hires_2d,unemployment_2d
Termination Date Month,,,
2019-09,19.655129,10.054448,3.567870
2019-10,20.165957,12.756465,3.562407
2019-11,20.375529,11.946757,3.574576
2019-12,20.085790,11.695120,3.576365


In [189]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: term')
accuracy_prod = forecast_accuracy(df_results['termination_2d'].values, df_test['termination'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

print('\nForecast Accuracy of: hire')
accuracy_prod = forecast_accuracy(df_results['hires_2d'].values, df_test['hires'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))
    
    
print('\nForecast Accuracy of: unemployment')
accuracy_prod = forecast_accuracy(df_results['unemployment_2d'].values, df_test['unemployment'])
for k, v in accuracy_prod.items():
    print(k, ': ', round(v,4))

Forecast Accuracy of: term
mape :  0.2293
me :  0.5706
mae :  4.1601
mpe :  0.0874
rmse :  4.7272
corr :  -0.7212
minmax :  0.1881

Forecast Accuracy of: hire
mape :  2.1265
me :  7.3632
mae :  7.3632
mpe :  2.1265
rmse :  7.7726
corr :  -0.791
minmax :  0.6206

Forecast Accuracy of: unemployment
mape :  0.2428
me :  -2.9797
mae :  3.0109
mpe :  -0.2339
rmse :  5.5772
corr :  0.6822
minmax :  0.2427
